# Выделение ключевых слов

__Идея:__ я хочу проверить, насколько хорошо будет работать матрица совстречаемости, которая будет основана на полном проходе по всему тексту (а не случайному выбору, пусть и большое количество раз, как в random walk, который мы обсуждали на парах).

_Дополнительная идея:_ посмотреть, как результат будет зависеть от величины «окна», которую мы зададим.

## Алгоритм

### Обработка текста

In [14]:
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

In [15]:
mystem = Mystem()
stops = stopwords.words("russian")
punctuators = punctuation + "«–»— \r\n"

In [16]:
def lemmatize_text(text):
    """Из исходного текста делает список лемм, очищенных от пунктуации
    и стоп-слов.
    
    :arg text (str): исходный текст, как он загружен из файла
    :returns clean_text (list of str): список лемм текста в том порядке, в
    котором они были в тексте
    """
    lemmas_raw = [parse["analysis"][0]["lex"] for parse in mystem.analyze(text)
                  if parse.get("analysis")]
    clean_text = [lemma_raw for lemma_raw in lemmas_raw
                 if lemma_raw not in stops and lemma_raw not in punctuators]        
    return clean_text

### Составление матрицы смежности

In [17]:
import numpy as np

In [18]:
def fill_co_occurence_matrix(text_lemmas, word_list, window):
    """Собирает матрицу совстречаемости из текста.
    
    :arg text_lemmas (list of str): лемматизированный и почищенный текст
    :arg word_list (list of str): список уникальных слов
    :arg window (int): окно, в котором мы считаем совстречаемость
    
    :returns M_co_occur (np.ndarray): матрица совстречаемости
    """
    M_co_occur = np.zeros((len(word_list), len(word_list)))
    for lemma_ind, lemma in enumerate(text_lemmas):
        lemma_matrix_index = word_list.index(lemma)
        ind_left = max(0, lemma_ind - window)
        ind_right = min(lemma_ind + window, len(text_lemmas))
        for ind_neighbor in range(ind_left, ind_right):
            word_neighbor = text_lemmas[ind_neighbor]
            neighbor_matrix_index = word_list.index(word_neighbor)
            M_co_occur[lemma_matrix_index][neighbor_matrix_index] += 1
    return M_co_occur

Предположим, что ключевыми будут те слова, которые соединяются с наибольшим количеством других слов => их совстречаемость будет самой высокой.

In [19]:
def extract_keywords_from_matrix(matrix, word_list, threshold):
    """Из матрицы совстречаемости выбирает сколько-то слов с наибольшей 
    совстречаемостью.
    
    :arg matrix (np.ndarray): матрица совстречаемости
    :arg word_list (list of str): список уникальных слов, на котором строилась 
    матрица
    :arg threshold (int): сколько первых значений совстречаемости брать
    
    :returns keywords (list of str): сами ключевые слова
    """
    kw_indices = np.where(np.argmax(matrix, axis=1) <= threshold)[0]
    keywords = [word_list[kw_ind] for kw_ind in kw_indices]
    return keywords

### Измеряем качество

In [26]:
def evaluate_kws(kws_true, kws_predicted):
    """Считает точность, полноту, F-меру и коэффициент Жаккара.
    
    :arg kws_true (list of str): размеченные вручную ключевые слова
    :arg kws_predicted (list of str): ключевые слова, найденные алгоритмом
    
    :returns precision, recall, f_score, jaccard (float): метрики
    """
    intersection = len(set(kws_true) & set(kws_predicted))
    precision = intersection / len(kws_predicted)
    recall = intersection / len(kws_true)
    try:
        f_score = (2 * precision * recall)/(precision + recall)
    except:
        f_score = 0
    jaccard = intersection / len(set(kws_true) | set(kws_predicted))
    return precision, recall, f_score, jaccard

### Всё вместе

In [21]:
def pipeline_for_texts(text, kw_true):
    """Единая функция-обёртка для работы с текстом от и до.
    
    :arg text (str): исходный текст
    :arg kw_true (list of str): приписанные вручную ключевые слова из датасета
    """
    text_lemmas = lemmatize_text(text)
    word_list = list(set(text_lemmas))
    for window in range(1, 6):
        print("window={}".format(window))
        M = fill_co_occurence_matrix(text_lemmas, word_list, window)
        for thresh in range(1, 4):
            kw = extract_keywords_from_matrix(M, word_list, thresh)
            print("\ttop-{} co-occurences: {}".format(thresh, kw))
            precision, recall, f1, jaccard = evaluate_kws(kw_true, kw)
            print("\tP={:.5f}, R={:.5f}, F1={:.5f}, J={:.5f}".format(precision, recall, f1, jaccard))

## Подгружаем данные из датасета

Датасет — [ru-kw-eval](https://github.com/mannefedov/ru_kw_eval_datasets), я взяла одну из Независимых газет (ха-ха).

In [22]:
import json

In [23]:
ng_data = []
with open("../data/ng_0.jsonlines", "r") as f:
    for line in f.readlines():
        ng_data.append(json.loads(line))

Возьмём какую-нибудь случайную статью и посмотрим, как работает:

In [24]:
import random

In [37]:
random.seed() 
article_ind = random.choice(range(0, len(ng_data)-1))
article_data = ng_data[article_ind]

print("TITLE: {}".format(article_data["title"]))
print("KEYWORDS: {}".format(article_data["keywords"]))

pipeline_for_texts(article_data["content"], article_data["keywords"])

TITLE: Хулиганы и витязи
KEYWORDS: ['сша', 'юмор', 'ирония', 'проза', 'эмиграция', 'провинция', 'еврея', 'тигр', 'монеты', 'макдональдс']
window=1
	top-1 co-occurences: ['заскакивать', 'какой-то', 'бумажка', 'доллар', 'следующий']
	P=0.00000, R=0.00000, F1=0.00000, J=0.00000
	top-2 co-occurences: ['заскакивать', 'какой-то', 'видение', 'бумажка', 'доллар', 'черт', 'следующий']
	P=0.00000, R=0.00000, F1=0.00000, J=0.00000
	top-3 co-occurences: ['заскакивать', 'какой-то', 'видение', 'удивляться', 'бумажка', 'доллар', 'черт', 'следующий', 'обилие']
	P=0.00000, R=0.00000, F1=0.00000, J=0.00000
window=2
	top-1 co-occurences: ['заскакивать', 'какой-то', 'бумажка', 'доллар', 'стыдно', 'красавица', 'неделя', 'следующий', 'бежать', 'печать']
	P=0.00000, R=0.00000, F1=0.00000, J=0.00000
	top-2 co-occurences: ['заскакивать', 'какой-то', 'видение', 'бумажка', 'доллар', 'стыдно', 'красавица', 'черт', 'неделя', 'просто', 'следующий', 'бежать', 'печать']
	P=0.00000, R=0.00000, F1=0.00000, J=0.00000
	t

## Выводы

1) Возможно, результаты такие ужасные, потому что в ключевых словах есть ещё и биграммы, которые не реализованы в этом алгоритме;

2) random walk не такой уж и плохой алгоритм;

3) новостные заметки небольшие, и поэтому совстречаемость может работать не очень хорошо, т.к. на малом объёме будет большая вариативность лемм, а чем текст больше, тем больше леммы повторяются.